In [242]:
import pandas as pd

# Lectura de la base de datos

In [255]:
df = pd.read_excel(r'data\base.xlsx',sheet_name = " Series a 61")
df_impuestos = pd.read_excel(r'data\base.xlsx',sheet_name = "Total")
new_names = ['Año','Q','SG01','SG02','SG03','SG04','SG05','SG06','SG07','SG08','SG09','SG10','SG11','SG12','SG13','SG14','SG15','SG16','SG17','SG18','SG19','SG20','SG21','SG22','SG23','SG24','SG25','SG26','SG27','SG28','SG29','SG30','SG31','SG32','SG33','SG34','SG35','SG36','SG37','SG38','SG39','SG40','SG41','SG42','SG43','SG44','SG45','SG46','SG47','SG48','SG49','SG50','SG51','SG52','SG53','SG54','SG55','SG56','SG57','SG58','SG59','SG60','SG61']
df.columns = new_names
df_clean = df.iloc[4:]

# informacion sobre las agrupaciones

In [244]:
# Niveles Agregaciones
nivel_25 = df.iloc[2:3]
nivel_25 = pd.melt(nivel_25,id_vars=["Año","Q"])
nivel_25.drop(columns=["Q","Año"],inplace=True)
nivel_25.rename(columns={'value':"ag_25"},inplace=True)
nivel_25.set_index("variable",inplace=True)

In [245]:
# Niveles Agregaciones
nivel_12 = df.iloc[3:4]
nivel_12 = pd.melt(nivel_12,id_vars=["Año","Q"])
nivel_12.drop(columns=["Q","Año"],inplace=True)
nivel_12.rename(columns={'value':"ag_12"},inplace=True)
nivel_12.set_index("variable",inplace=True)

# Bases para el cálculo segun el nivel de agregación

In [296]:
df_clean_collapsed = pd.melt(df_clean,id_vars=["Año","Q"])
df_clean_collapsed.set_index("variable",inplace=True)
#joins con los niveles
ndf_clean_f = df_clean_collapsed.join(nivel_25,how="left",rsuffix="_")
ndf_clean_f = ndf_clean_f.join(nivel_12,how="left",rsuffix="_1")

In [297]:
#agregacion total
agg_total = df_clean_collapsed.groupby(["Año","Q"]).sum().reset_index()

In [247]:
ndf_clean_f_25 = ndf_clean_f.drop(columns="ag_12")
ndf_clean_f_12 = ndf_clean_f.drop(columns="ag_25")
data_agg_nv25 = ndf_clean_f_25.groupby(["Año","Q","ag_25"]).sum().reset_index()
data_agg_nv12 = ndf_clean_f_12.groupby(["Año","Q","ag_12"]).sum().reset_index()

In [248]:
# agregracion a nivel 25
new_data_agg_nv25 = data_agg_nv25.pivot(index=["Año","Q"], columns="ag_25")
new_data_agg_nv25.columns = ['_'.join(col) for col in new_data_agg_nv25.columns]
new_data_agg_nv25.reset_index(inplace=True)

In [252]:
# agregracion a nivel 12
new_data_agg_nv12 = data_agg_nv12.pivot(index=["Año","Q"], columns="ag_12")
new_data_agg_nv12.columns = ['_'.join(col) for col in new_data_agg_nv12.columns.map(str)]
new_data_agg_nv12.reset_index(inplace=True)
nuevas_ = ["value_"+str(number) for number in range(1,13)]
elem = ["Año","Q"]
elem.extend(nuevas_)
new_data_agg_nv12.columns = elem
new_data_agg_nv12

In [285]:
# Niveles Agregaciones
df_impuestos_clean = df_impuestos.iloc[4:]
names_df_impuestos_clean = ["Año","Q","VAB","TX","PIB"]
df_impuestos_clean.columns = names_df_impuestos_clean
df_impuestos_clean = df_impuestos_clean[["Año","Q","TX"]]
df_impuestos_clean

,Año,Q,TX
4,2005,1,10116.975394
5,2005,2,10557.154372
6,2005,3,10747.573036
7,2005,4,11103.297197
8,2006,1,11537.380003
...,...,...,...
66,2020,3,18770.953236
67,2020,4,20125.764542
68,2021,1,20683.535937
69,2021,2,21288.150668


# Cálculo del PIB

In [301]:
final_df_pib = agg_total.set_index(["Año","Q"])
df_tx = df_impuestos_clean.set_index(["Año","Q"])

In [312]:
pib = final_df_pib.join(df_tx)
pib.reset_index(inplace=True)
pib.rename(columns={"value":"VAB"},inplace=True)

In [314]:
pib["PIB"] = pib["VAB"]+pib["TX"]
pib

,Año,Q,VAB,TX,PIB
0,2005,1,118434.185533,10116.975394,128551.160927
1,2005,2,119141.617273,10557.154372,129698.771644
2,2005,3,119169.939243,10747.573036,129917.512279
3,2005,4,121013.257914,11103.297197,132116.555111
4,2006,1,124199.286280,11537.380003,135736.666282
...,...,...,...,...,...
62,2020,3,183660.200116,18770.953236,202431.153352
63,2020,4,193603.424821,20125.764542,213729.189362
64,2021,1,201045.923216,20683.535937,221729.459154
65,2021,2,196732.954353,21288.150668,218021.105021


# Variacion Anual 2019

In [284]:
Serie_transporte = new_data_agg_nv25[["Año","Q","value_GR15"]]
Serie_transporte_2019 = Serie_transporte[(Serie_transporte["Año"].isin([2019,2018]))&(Serie_transporte["Q"]==3)]
Serie_transporte_2019
value_2018=Serie_transporte_2019["value_GR15"].iloc[0]
value_2019=Serie_transporte_2019["value_GR15"].iloc[1]
variacion_anual = round((value_2019-value_2018)/value_2019*100,2)
print("La variación anual para el 2019 fue de: ",variacion_anual,"%")


La variación anual para el 2019 fue de:  5.52 %


# Exporta resultados

In [320]:
#agragacion a 25
new_data_agg_nv25.to_csv("resultados/agg25.csv",index=False)
#agragacion a 12
new_data_agg_nv12.to_csv("resultados/agg12.csv",index=False)
# base del PIB
pib.to_csv("resultados/pib.csv",index=False)